In [1]:
import torch

# "in-place" operation 

In [7]:
x = torch.rand(2, 3)
y = torch.rand_like(x)

print(x)
print(y)

tensor([[0.5145, 0.9428, 0.4033],
        [0.0284, 0.0097, 0.3923]])
tensor([[0.1324, 0.3479, 0.2404],
        [0.4318, 0.2268, 0.9512]])


In [9]:
z1 = torch.add(x, y)
z1

tensor([[0.6469, 1.2908, 0.6436],
        [0.4602, 0.2365, 1.3434]])

In [11]:
z2 = torch.empty_like(x)
print(z2)
torch.add(x, y, out=z2)
print(z2)

tensor([[ 1.4013e-45,  4.5818e-41,  0.0000e+00],
        [ 0.0000e+00, -2.5310e+24,  4.5818e-41]])
tensor([[0.6469, 1.2908, 0.6436],
        [0.4602, 0.2365, 1.3434]])


In [12]:
y.add_(x)
y

tensor([[0.6469, 1.2908, 0.6436],
        [0.4602, 0.2365, 1.3434]])

# All about "autograd"

## Tensor autograd functions

### .data vs .detach 

In [65]:
a = torch.tensor([1, 2, 3.], requires_grad=True)

out = a.sigmoid()

b = out.data
b.zero_()

tensor([0., 0., 0.])

In [66]:
out  # out  was modified by c.zero_()

tensor([0., 0., 0.], grad_fn=<SigmoidBackward>)

In [67]:
out.sum().backward()

In [68]:
a.grad  # The result is very, very wrong because `out` changed!

tensor([0., 0., 0.])

In [69]:
hex(id(a))

'0x7fb9a84ebdc8'

In [70]:
hex(id(b))

'0x7fb9a84eb5a0'

In [71]:
a = torch.tensor([1, 2, 3.], requires_grad=True)

out = a.sigmoid()

c = out.detach()
c.zero_()

tensor([0., 0., 0.])

In [72]:
out  # modified by c.zero_() !!

tensor([0., 0., 0.], grad_fn=<SigmoidBackward>)

In [73]:
out.sum().backward()  # Requires the original value of out, but that was overwritten by c.zero_()

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [3]], which is output 0 of SigmoidBackward, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [74]:
a.grad

In [75]:
hex(id(a))

'0x7fb9c8ce11f8'

In [76]:
hex(id(c))

'0x7fb9c8d278b8'

## Functions

## Locally disabling gradient computation

In [34]:
x = torch.tensor([1.], requires_grad=True)

with torch.no_grad():
    y = x * 2
y.requires_grad

False

In [35]:
@torch.no_grad()
def doubler(x):
    return x * 2

z = doubler(x)
z.requires_grad

False

In [45]:
x = torch.tensor([1.], requires_grad=True)

with torch.no_grad():
    with torch.enable_grad():
        y = x * 2
y.requires_grad

True

In [46]:
y.backward()
x.grad

tensor([2.])

In [47]:
@torch.enable_grad()
def doubler(x):
    return x * 2

z = doubler(x)
z.requires_grad

True